# Speech Recognition Game

AI plays against a human, who tries to guess a word (or enter a voice command).<br>
We experiment with 
- the use of microphone
- transcribtion of spoken language
by use of Google API

Response: a dictionary with three keys:
- response["success"]: a boolean indicating whether or not the API request was successful
- response["error"]: `None` if no error occured, otherwise a string containing an error message if the API could not be reached orbspeech was unrecognizable
- response["transcription"]: a string containing the transcribed text or `None` if speech could not be transcribed

In [ ]:
import random
import time
import speech_recognition as sr

In [55]:
# set the list of words, maxnumber of guesses, and prompt limit
WORDS = ["apple", "banana", "grape", "orange", "mango", "lemon"]
NUM_GUESSES = 2
PROMPT_LIMIT = 2

In [47]:
# set up the response object
response = {
        "success": True,
        "error": None,
        "transcription": None
}

In [48]:
# create recognizer and mic instances
recognizer = sr.Recognizer()
microphone = sr.Microphone()

In [49]:
# check that recognizer and microphone arguments are appropriate type
if not isinstance(recognizer, sr.Recognizer):
        raise TypeError("`recognizer` must be `Recognizer` instance")

if not isinstance(microphone, sr.Microphone):
        raise TypeError("`microphone` must be `Microphone` instance")

In [50]:
# Transcribe speech from recorded from `microphone`
def recognize_speech_from_mic(recognizer, microphone):
    
    # adjust the recognizer sensitivity to ambient noise and record audio from the microphone
    with microphone as source:
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)

    
    # try recognizing the speech in the recording
    # if a RequestError or UnknownValueError exception is caught, update the response object accordingly
    try:
        response["transcription"] = recognizer.recognize_google(audio)
    except sr.RequestError:
        # API was unreachable or unresponsive
        response["success"] = False
        response["error"] = "API unavailable"
    except sr.UnknownValueError:
        # speech was unintelligible
        response["error"] = "Unable to recognize speech"

    return response


## AI Play

In [56]:
# get a random word from the list
word = random.choice(WORDS)
word

'lemon'

In [57]:
# print instructions
instructions = ( "I'm thinking of one of these words: {words}. You have {n} tries to guess which one.\n"
               ).format(words=', '.join(WORDS), n = NUM_GUESSES)

In [58]:
# show instructions and wait 3 seconds before starting the game
print(instructions)
time.sleep(3)

I'm thinking of one of these words: apple, banana, grape, orange, mango, lemon. You have 2 tries to guess which one.



In [60]:
for i in range(NUM_GUESSES):
    for j in range(PROMPT_LIMIT):
        print('Guess {}. Speak!'.format(i+1))
        guess = recognize_speech_from_mic(recognizer, microphone)
        
        if guess["transcription"]:
                break
        if not guess["success"]:
                break
        print("I didn't catch that. What did you say?\n")

        # if there was an error, stop the game
        if guess["error"]:
            print("ERROR: {}".format(guess["error"]))
            break

        # show the user the transcription
        print("You said: {}".format(guess["transcription"]))

    # determine if guess is correct and if any attempts remain
    guess_is_correct = guess["transcription"].lower() == word.lower()
    user_has_more_attempts = i < NUM_GUESSES - 1

        # determine if the user has won the game
        # if not, repeat the loop if user has more attempts
        # if no attempts left, the user loses the game
    if guess_is_correct:
            print("Correct! You win!")
            break
    elif user_has_more_attempts:
            print("Incorrect. Try again.\n")
    else:
            print("Sorry, you lose! I was thinking of '{}'.".format(word))
            break


Guess 1. Speak!
result2:
[]
Incorrect. Try again.

Guess 2. Speak!
result2:
{   'alternative': [   {'confidence': 0.87658823, 'transcript': 'apple'},
                       {'transcript': 'Airport'},
                       {'transcript': 'April'},
                       {'transcript': 'purple'}],
    'final': True}
Sorry, you lose! I was thinking of 'lemon'.


## Reference
https://realpython.com/python-speech-recognition/